In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

d:\code\VSCode\DS project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('laptop_merged_v1_fix.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353 entries, 0 to 1352
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cam_ung              1353 non-null   int64  
 1   height_mm            1263 non-null   float64
 2   url_path             1353 non-null   object 
 3   material             1353 non-null   object 
 4   storage_max_support  1353 non-null   float64
 5   storage_gb           1353 non-null   float64
 6   display_width        1353 non-null   float64
 7   bluetooth_version    1353 non-null   float64
 8   width_mm             1270 non-null   float64
 9   manufacturer         1353 non-null   object 
 10  cpu_max_speed        936 non-null    float64
 11  root_price           1000 non-null   float64
 12  cpu_threads          711 non-null    float64
 13  cpu_cores            1193 non-null   float64
 14  ram_speed            1203 non-null   float64
 15  product_weight       1353 non-null   f

In [5]:
df.head()

,cam_ung,height_mm,url_path,material,storage_max_support,storage_gb,display_width,bluetooth_version,width_mm,manufacturer,cpu_max_speed,root_price,cpu_threads,cpu_cores,ram_speed,product_weight,discounted_price,ram_type,refresh_rate,cpu_speed,name,ram_storage,is_installment,ram_slots,os_version,battery_capacity,laptop_color,cpu_model,nhu_cau_su_dung,vga_type,depth_mm,image,vga_brand,product_id,cpu_series,display_height,vga_vram,laptop_camera,display_size,cpu_brand
0,0,227.0,https://www.thegioididong.com/laptop/asus-fa40...,vỏ nhựa - nắp lưng kim loại,2048.0,1024.0,2560.0,5.3,311.0,asus,5.1,44990000.0,24.0,12.0,7500.0,1.46,39990000.0,lpddr5,165.0,2.0,Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 37...,16.0,1,0.0,windows 11,73.0,xám,hx 370,"['gaming', 'đồ họa - kỹ thuật', 'laptop sáng t...",card rời,19.90,https://cdn.tgdd.vn/Products/Images/44/328948/...,nvidia,0220042002813__l265_20250402-105034,ryzen ai 9,1600.0,8.0,full hd,14.0,amd
1,0,235.0,https://www.thegioididong.com/laptop/lenovo-id...,vỏ nhựa,1024.0,1024.0,1920.0,5.2,359.3,lenovo,4.9,19490000.0,16.0,10.0,4800.0,1.62,18190000.0,lpddr5,60.0,2.4,Laptop Lenovo Ideapad Slim 3 15IRH8 i7 13620H/...,16.0,1,0.0,windows 11,47.0,xám,13620h,"['gaming', 'học tập - văn phòng', 'đồ họa - kỹ...",card tích hợp,17.90,https://cdn.tgdd.vn/Products/Images/44/322073/...,intel,0220042002563__l66_20250402-105034,core i7,1080.0,NaN,hd,15.6,intel
2,0,237.5,https://www.thegioididong.com/laptop/acer-aspi...,vỏ nhựa,1024.0,512.0,1920.0,5.1,362.9,acer,4.5,NaN,8.0,6.0,4800.0,1.70,NaN,lpddr5,60.0,1.2,Laptop Acer Aspire 5 A515 58P 34RJ i3 1315U/8G...,8.0,1,0.0,windows 11,50.0,xám,1315u,"['cao cấp - sang trọng', 'học tập - văn phòng']",card tích hợp,17.99,https://cdn.tgdd.vn/Products/Images/44/307185/...,intel,0220042002272__l46_20250402-105034,core i3,1080.0,NaN,hd,15.6,intel
3,0,232.5,https://www.thegioididong.com/laptop/asus-vivo...,vỏ nhựa,512.0,512.0,1920.0,5.3,359.7,asus,4.4,12290000.0,8.0,6.0,3200.0,1.70,10690000.0,ddr4,60.0,1.2,Laptop Asus Vivobook 15 X1504ZA i3 1215U/8GB/5...,8.0,1,2.0,windows 11,42.0,bạc,1215u,"['cao cấp - sang trọng', 'gaming', 'học tập - ...",card tích hợp,17.90,https://cdn.tgdd.vn/Products/Images/44/312414/...,intel,0220042002367__l9_20250402-105034,core i3,1080.0,NaN,hd,15.6,intel
4,0,212.4,https://www.thegioididong.com/laptop/apple-mac...,vỏ kim loại,256.0,256.0,2560.0,5.0,304.1,apple,3.2,19990000.0,NaN,8.0,4266.0,1.29,17190000.0,NaN,60.0,NaN,Laptop MacBook Air 13 inch M1 8GB/256GB (MGN63...,8.0,1,0.0,macos,NaN,xám,apple m1,"['cao cấp - sang trọng', 'học tập - văn phòng'...",card tích hợp,4.10,https://cdn.tgdd.vn/Products/Images/44/231244/...,apple,0220042001279__l4_20250402-105034,m1,1600.0,NaN,hd,13.3,apple


In [6]:
df_1 = df[df['url_path'].str.contains("https://www.thegioididong.com", na=False)]
df_2 = df[~df['url_path'].str.contains("https://www.thegioididong.com", na=False)]

In [ ]:
# Bỏ qua các cột không cần so sánh
EXCLUDE_COLS = ['url_path', 'product_id', 'image', 'name', 'nhu_cau_su_dung']

# Xác định các cột cần dùng để tạo mô tả
columns_to_compare = [
    col for col in df.columns
    if col not in EXCLUDE_COLS and df[col].dtype in ['float64', 'int64', 'object']
]

def row_to_string(row):
    """
    Chuyển một dòng dữ liệu thành chuỗi mô tả gộp lại từ các cột đã chọn.
    """
    return ' | '.join(
        f"{col}: {str(row[col])}" 
        for col in columns_to_compare 
        if pd.notnull(row[col])
    )

# Tạo chuỗi mô tả cho mỗi dòng
df_1['row_text'] = df_1.apply(row_to_string, axis=1)
df_2['row_text'] = df_2.apply(row_to_string, axis=1)

# Load mô hình embedding
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

# Vector hóa văn bản
embeddings_1 = model.encode(df_1['row_text'].tolist(), show_progress_bar=True)
embeddings_2 = model.encode(df_2['row_text'].tolist(), show_progress_bar=True)

# Gán nhãn dựa trên similarity
df_1['nhu_cau_su_dung_gan'] = None
df_1['similarity_score'] = 0.0

for i in tqdm(range(len(df_1))):
    sims = cosine_similarity([embeddings_1[i]], embeddings_2)[0]
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]

    if best_score > 0.9:  # Ngưỡng có thể điều chỉnh tùy mục tiêu
        df_1.at[i, 'nhu_cau_su_dung_gan'] = df_2.iloc[best_idx]['nhu_cau_su_dung']
        df_1.at[i, 'similarity_score'] = best_score

# Kiểm tra độ trùng nhãn
df_1['same_label'] = df_1['nhu_cau_su_dung'] == df_1['nhu_cau_su_dung_gan']

C:\Users\ACER\AppData\Local\Temp\ipykernel_16536\4222820429.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['row_text'] = df_1.apply(row_to_string, axis=1)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16536\4222820429.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['row_text'] = df_2.apply(row_to_string, axis=1)
d:\code\VSCode\DS project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store du

In [19]:
df_1['nhu_cau_su_dung'] = df_1['nhu_cau_su_dung_gan']

C:\Users\ACER\AppData\Local\Temp\ipykernel_16536\3976746692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['nhu_cau_su_dung'] = df_1['nhu_cau_su_dung_gan']


In [21]:
df_1 = df_1.drop(columns= 'nhu_cau_su_dung_gan')

In [22]:
df = pd.concat([df_1, df_2], ignore_index=True)